### GraphSVX

Here we visualize the explainer results and analyze specific graph theoretic properties for datatset ```syn1```

In [1]:
# imports
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
results = '../results/node_masks/'
# Load produced masks
dirs = os.listdir(results)
dirs

['node_mask_syn1_explain_node_400.npy',
 'node_mask_syn1_explain_node_405.npy',
 'node_mask_syn1_explain_node_410.npy',
 'node_mask_syn1_explain_node_415.npy',
 'node_mask_syn1_explain_node_420.npy',
 'node_mask_syn1_explain_node_425.npy',
 'node_mask_syn1_explain_node_430.npy',
 'node_mask_syn1_explain_node_435.npy',
 'node_mask_syn1_explain_node_440.npy',
 'node_mask_syn1_explain_node_445.npy',
 'node_mask_syn1_explain_node_450.npy',
 'node_mask_syn1_explain_node_455.npy',
 'node_mask_syn1_explain_node_460.npy',
 'node_mask_syn1_explain_node_465.npy',
 'node_mask_syn1_explain_node_470.npy',
 'node_mask_syn1_explain_node_475.npy',
 'node_mask_syn1_explain_node_480.npy',
 'node_mask_syn1_explain_node_485.npy',
 'node_mask_syn1_explain_node_490.npy',
 'node_mask_syn1_explain_node_495.npy',
 'node_mask_syn1_explain_node_500.npy',
 'node_mask_syn1_explain_node_505.npy',
 'node_mask_syn1_explain_node_510.npy',
 'node_mask_syn1_explain_node_515.npy',
 'node_mask_syn1_explain_node_520.npy',


In [3]:
# Get the masks
masks = []
# This would print all the files and directories
for file in dirs:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == 'syn1':
        # print(file)
        masks.append(file)
len(masks)

60

In [4]:
def show_adjacency_full(mask):
    """Obtain the numpy array of the mask

    Args:
        mask (str): Filename containing the mask
        ax (Axes class object, optional): Axis. Defaults to None.

    Returns:
        numpy array: Numpy array version of the mask
    """
    # Obtain mask as array from the filename sent in var "mask"
    adj = np.load(os.path.join(results, mask), allow_pickle=True)

    return adj

### Analyzing Graph Properties

Here we measure the following quantities:

- Nodes: |V|
- Edges: |E|
- Avg degree
- Diameter of the graph
- Sparsity measure: edge density = |E|/C(|V|,2)
- Node Betweenness centrality

In [5]:
# Get data adjacency matrix
datadir = '../results/data_adj/'
matrices = os.listdir(datadir)
matrices

['adj_matrix_syn1.npy', 'adj_matrix_syn2.npy', 'adj_matrix_syn4.npy']

In [6]:
data = []
dataset = 'syn1.npy'
# This would print all the files and directories
for file in matrices:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == dataset:
        data.append(file)
data

['adj_matrix_syn1.npy']

In [7]:
adj_matrix = np.load(os.path.join(datadir, data[0]), allow_pickle=True)
adj_matrix.shape

(700, 700)

In [15]:
def normalize(A):
    scale_factor = A.max() - A.min()
    B = np.ones_like(A)*A.min()

    A = (A - B)/scale_factor
    return A

def centrality(G,mask,node_to_explain):
    
    sources = [node_to_explain]
    targets = [v for v in list(G.nodes()) if v!=node_to_explain]
    bc = nx.betweenness_centrality_subset(G,sources,targets)

    # By explainer
    A = mask.copy()
    print(len(G))
    print(len(A))

def graph_prop(graph,mask,node_to_explain):

    v = graph.number_of_nodes()
    e = graph.number_of_edges()
    avg_degree = float('%.3f'%(2*e/v))
    diameter = nx.diameter(graph)
    sparsity = float('%.3f'%(2*e/(v*(v-1))))
    centrality(graph,mask,node_to_explain)

    return [v,e,avg_degree,diameter,sparsity]

In [16]:
def get_node_to_explain(filename):

    required_slice = filename.split('_')[-1]
    node = int(required_slice.split('.')[0])

    return node

In [17]:
graph_properties = []

# for m in masks:
#     mask = show_adjacency_full(m)
#     G = nx.from_numpy_array(adj_matrix)
#     node_to_explain = get_node_to_explain(m)

#     prop = graph_prop(G,mask,node_to_explain)

mask = show_adjacency_full(masks[0])
G = nx.from_numpy_array(adj_matrix)
node_to_explain = get_node_to_explain(masks[0])

prop = graph_prop(G,mask,node_to_explain)
prop

700
700


[700, 2055, 5.871, 10, 0.008]